In [28]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv("on_resto_chinese_tokens.csv", engine="python")
df.head()

,user_id,business_id,stars,text,state,rating,categories,star_categories,tokens
0,jlu4CztcSxrKx56ba1a5AQ,3fw2X5bZYeW9xCz_zGhOHg,3.0,Tracy dessert had a big name in Hong Kong and ...,ON,3.5,"Food, Chinese, Restaurants, Desserts",Bad,"['tracy', 'dessert', 'big', 'hong', 'kong', 'm..."
1,nkSf1NKWFGiAyrnCa-A5UA,3fw2X5bZYeW9xCz_zGhOHg,3.0,The squid balls were AMAZING. Crispy exterior ...,ON,3.5,"Food, Chinese, Restaurants, Desserts",Bad,"['squid', 'ball', 'amazing', 'crispy', 'exteri..."
2,jG7130OQWiftvHhtyabsog,3fw2X5bZYeW9xCz_zGhOHg,4.0,"Must Try: Mango Sago with Pommels, Shaved Ice ...",ON,3.5,"Food, Chinese, Restaurants, Desserts",Good,"['try', 'mango', 'sago', 'pommel', 'shave', 'i..."
3,wdCwBv_TA_Y-IjUZv2HWWQ,3fw2X5bZYeW9xCz_zGhOHg,1.0,We came on a Friday and the store said that th...,ON,3.5,"Food, Chinese, Restaurants, Desserts",Bad,"['come', 'friday', 'store', 'say', 'open', 'pm..."
4,kxz6Q2AERQeo9x6nnbHZNg,3fw2X5bZYeW9xCz_zGhOHg,3.0,I think this is a cool little hang out place f...,ON,3.5,"Food, Chinese, Restaurants, Desserts",Bad,"['think', 'cool', 'little', 'hang', 'place', '..."


In [77]:
token_len = [len(token_list) for token_list in df['tokens']]

In [82]:
from scipy import stats
stats.describe(np.array(token_len))

DescribeResult(nobs=51621, minmax=(2, 5607), mean=528.6434396853994, variance=191404.27758912923, skewness=2.1994617994550083, kurtosis=8.050708549409517)

In [86]:
import math
#Get the 75th percentile
528 + (0.674)*(191404)**(0.5)

822.8732668520495

In [90]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

# Build LSTM
max_words = 20000
maxlen=823
batch_size = 32
model = Sequential()
model.add(Embedding(max_words, 64, input_length=maxlen))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 823, 64)           1280000   
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 1,313,089
Trainable params: 1,313,089
Non-trainable params: 0
_________________________________________________________________


In [91]:
#Split Test/Train
from sklearn.model_selection import train_test_split 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(df['tokens'])

df['tokens']= df['tokens'].astype(str)

train, test = train_test_split(df, test_size=0.33, random_state=42)

feature = 'tokens'
target = 'star_categories'

#Train
X_train = train[feature]
#Convert X_train to sequence
X_train = tokenizer.texts_to_sequences(X_train)
#Pad sequence to max_len
X_train = pad_sequences(X_train, maxlen=maxlen)
#Convert Good/Bad to 1/0 and to array
y_train = np.array([ 1 if target =='Good' else 0 for target in train[target]])

# Validation Sets
X_test = test[feature]
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=maxlen)
y_test = np.array([ 1 if target =='Good' else 0 for target in test[target]])

In [ ]:
unicorns = model.fit(X_train, y_train,
          batch_size=batch_size, 
          epochs=5, 
          validation_data=(X_test,y_test))

Epoch 1/5
1081/1081 [==============================] - 1202s 1s/step - loss: 0.4597 - acc: 0.7897 - val_loss: 0.4006 - val_acc: 0.8190
Epoch 2/5
1081/1081 [==============================] - 1212s 1s/step - loss: 0.3798 - acc: 0.8321 - val_loss: 0.4027 - val_acc: 0.8181
Epoch 3/5
1081/1081 [==============================] - 1220s 1s/step - loss: 0.3581 - acc: 0.8437 - val_loss: 0.3925 - val_acc: 0.8237
Epoch 4/5
1062/1081 [============================>.] - ETA: 21s - loss: 0.3379 - acc: 0.8539

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss values
plt.plot(unicorns.history['acc'])
plt.plot(unicorns.history['val_acc'])
plt.title('Model loss')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show();